## Update metric definitions and specifications in SQuaSH

Set the API instance. For the demo instance:

In [ ]:
SQUASH_API_URL = "https://squash-restful-api-demo.lsst.codes"

### Authenticate user

In [ ]:
import getpass
username = 'nobody'
password = getpass.getpass(prompt='Password for user `{}`: '.format(username))

In [ ]:
import requests

user={'username': username, 'password': password}
r = requests.post(SQUASH_API_URL + "/register", json=user)
r.json()


In [ ]:
r = requests.post(SQUASH_API_URL + "/auth", json=user)
r.json()

Add the access token into the authorization header for the next requests.

In [ ]:
headers = dict()
headers['Authorization'] = 'JWT ' + r.json()['access_token']

### Get metric definitions and specifications from the verify_metrics repository

In [ ]:
%%bash
git clone https://github.com/lsst/verify_metrics.git

In [ ]:
import lsst.verify
metrics = lsst.verify.MetricSet.load_metrics_package('verify_metrics')
specs = lsst.verify.SpecificationSet.load_metrics_package('verify_metrics')

### Update metric definitions

In [ ]:

for metric in metrics.json:
    r = requests.put(SQUASH_API_URL + "/metric/{}".format(metric['name']), json=metric, headers=headers)
    if r.status_code == requests.codes.not_found:
        print('Metric {} not found, creating...'.format(metric))
        r = requests.post(SQUASH_API_URL + "/metric/{}".format(metric['name']), json=metric, headers=headers)
        

### Update specifications

In [ ]:
for spec in specs.json:
    r = requests.put(SQUASH_API_URL + "/spec/{}".format(spec['name']), json=spec, headers=headers)
    if r.status_code == requests.codes.not_found:
        print('Specification {} not found, creating...'.format(spec))
        r = requests.post(SQUASH_API_URL + "/spec/{}".format(spec['name']), json=spec, headers=headers)

